# Day 15: 画像の統計解析

## Learning Objectives
- 画像の統計量を計算する（平均、分散、ヒストグラム）
- 画像品質評価指標を理解する
- 統計的画像処理を実践する

## 1. 画像統計量の計算

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import math

In [ ]:
def calculate_image_stats(image):
    """画像の統計量を計算する"""
    # 平均値
    mean = np.mean(image)
    
    # 中央値
    median = np.median(image)
    
    # 標準偏差
    std = np.std(image)
    
    # 最小値と最大値
    min_val = np.min(image)
    max_val = np.max(image)
    
    # 画像サイズ
    height, width = image.shape
    
    return {
        'mean': mean,
        'median': median,
        'std': std,
        'min': min_val,
        'max': max_val,
        'height': height,
        'width': width
    }

# テスト用の画像を作成
test_image = np.array([
    [100, 110, 120, 130, 140],
    [110, 120, 130, 140, 150],
    [120, 130, 140, 150, 160],
    [130, 140, 150, 160, 170],
    [140, 150, 160, 170, 180]
], dtype=np.uint8)

print("テスト画像:")
print(test_image)
print()

# 統計量を計算
stats = calculate_image_stats(test_image)
for key, value in stats.items():
    print(f"{key}: {value}")

In [ ]:
def calculate_histogram(image, bins=256):
    """画像のヒストグラムを計算する"""
    # 1次元配列に変換
    flat_image = image.flatten()
    
    # ヒストグラムを計算
    hist, bin_edges = np.histogram(flat_image, bins=bins, range=(0, 256))
    
    return hist, bin_edges

# ヒストグラムを表示
def plot_histogram(image, title="ヒストグラム"):
    """ヒストグラムをプロットする"""
    hist, bin_edges = calculate_histogram(image)
    
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.imshow(image, cmap='gray', vmin=0, vmax=255)
    plt.title(title)
    plt.axis('off')
    
    plt.subplot(1, 2, 2)
    plt.bar(bin_edges[:-1], hist, width=1)
    plt.title("ヒストグラム")
    plt.xlabel("画素値")
    plt.ylabel("ピクセル数")
    plt.show()

# テスト
print("ヒストグラムの計算と表示:")
plot_histogram(test_image, "テスト画像")

## 2. 画像品質評価指標

In [ ]:
def calculate_psnr(original, compressed):
    """PSNR (Peak Signal-to-Noise Ratio) を計算する"""
    mse = np.mean((original - compressed) ** 2)
    if mse == 0:
        return float('inf')
    max_pixel = 255.0
    psnr = 20 * np.log10(max_pixel / np.sqrt(mse))
    return psnr

def calculate_ssim(image1, image2, k1=0.01, k2=0.03, L=255):
    """SSIM (Structural Similarity) を簡易的に計算する"""
    # 平均と分散
    mu1 = np.mean(image1)
    mu2 = np.mean(image2)
    sigma1 = np.std(image1)
    sigma2 = np.std(image2)
    
    # 共分散
    sigma12 = np.cov(image1.flatten(), image2.flatten())[0, 1]
    
    # SSIM計算
    c1 = (k1 * L) ** 2
    c2 = (k2 * L) ** 2
    
    numerator = (2 * mu1 * mu2 + c1) * (2 * sigma12 + c2)
    denominator = (mu1 ** 2 + mu2 ** 2 + c1) * (sigma1 ** 2 + sigma2 ** 2 + c2)
    
    return numerator / denominator

# ノイズ付き画像を作成
noise = np.random.normal(0, 20, test_image.shape)
noisy_image = np.clip(test_image.astype(float) + noise, 0, 255).astype(np.uint8)

# PSNRとSSIMを計算
psnr_value = calculate_psnr(test_image, noisy_image)
ssim_value = calculate_ssim(test_image, noisy_image)

print(f"PSNR: {psnr_value:.2f} dB")
print(f"SSIM: {ssim_value:.4f}")

# 可視化
plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.imshow(test_image, cmap='gray', vmin=0, vmax=255)
plt.title("オリジナル")
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(noisy_image, cmap='gray', vmin=0, vmax=255)
plt.title("ノイズ付き")
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(test_image - noisy_image, cmap='gray', vmin=-50, vmax=50)
plt.title("差分")
plt.axis('off')

plt.tight_layout()
plt.show()

## 3. 統計的画像処理

In [ ]:
def histogram_equalization(image):
    """ヒストグラム平坦化を行う"""
    # ヒストグラムを計算
    hist, _ = calculate_histogram(image)
    
    # 累積分布関数を計算
    cdf = hist.cumsum()
    
    # 正規化（0-255にスケール）
    cdf_normalized = cdf * 255 / cdf[-1]
    
    # ルックアップテーブルを作成
    lookup_table = np.round(cdf_normalized).astype(np.uint8)
    
    # ルックアップテーブルを適用
    equalized = lookup_table[image]
    
    return equalized

# ヒストグラム平坦化を実行
equalized = histogram_equalization(test_image)

# 結果を表示
print("ヒストグラム平坦化の効果:")
print(f"オリジナル画像の統計量:")
stats_original = calculate_image_stats(test_image)
for key, value in stats_original.items():
    print(f"  {key}: {value}")

print(f"\n平坦化後の統計量:")
stats_equalized = calculate_image_stats(equalized)
for key, value in stats_equalized.items():
    print(f"  {key}: {value}")

In [ ]:
# 画像全体の比較
plt.figure(figsize=(15, 10))

# オリジナル
plt.subplot(2, 3, 1)
plt.imshow(test_image, cmap='gray', vmin=0, vmax=255)
plt.title("オリジナル画像")
plt.axis('off')
plot_histogram(test_image, "ヒストグラム")

# 平坦化後
plt.subplot(2, 3, 4)
plt.imshow(equalized, cmap='gray', vmin=0, vmax=255)
plt.title("ヒストグラム平坦化後")
plt.axis('off')
plot_histogram(equalized, "ヒストグラム")

# しきい値処理
threshold = 130
binary = (test_image > threshold).astype(np.uint8) * 255
plt.subplot(2, 3, 2)
plt.imshow(binary, cmap='gray', vmin=0, vmax=255)
plt.title(f"しきい値処理 (>{threshold})")
plt.axis('off')

# 平滑化（フィルタ）
smoothed = cv2.GaussianBlur(test_image, (3, 3), 0)
plt.subplot(2, 3, 3)
plt.imshow(smoothed, cmap='gray', vmin=0, vmax=255)
plt.title("ガウシアン平滑化")
plt.axis('off')
plot_histogram(smoothed, "ヒストグラム")

plt.tight_layout()
plt.show()

## 実践演習

### 演習1: 統計量の計算
- 異なるパターンの画像を作成し、統計量を比較してみましょう
- 画素値の分布がどう統計量に影響するかを観察する

In [ ]:
# 異なるパターンの画像を作成
# 一様分布の画像
uniform_image = np.random.randint(0, 256, (100, 100), dtype=np.uint8)

# 正規分布の画像
normal_image = np.clip(np.random.normal(128, 30, (100, 100)), 0, 255).astype(np.uint8)

# ノイズの少ない画像
smooth_image = np.ones((100, 100), dtype=np.uint8) * 128
smooth_image[20:80, 20:80] = 200

# 統計量を比較
images = [uniform_image, normal_image, smooth_image]
names = ["一様分布", "正規分布", "ノイズの少ない画像"]

print("異なるパターンの画像の統計量比較:")
for i, (img, name) in enumerate(zip(images, names)):
    print(f"\n{name}:")
    stats = calculate_image_stats(img)
    for key, value in stats.items():
        print(f"  {key}: {value:.2f}")

### 演習2: ノイズ除去
- 統計的処理を使ってノイズを除去してみましょう

In [ ]:
def add_noise(image, noise_level=20):
    """画像にノイズを加える"""
    noise = np.random.normal(0, noise_level, image.shape)
    noisy = np.clip(image.astype(float) + noise, 0, 255).astype(np.uint8)
    return noisy

def median_filter(image, kernel_size=3):
    """メディアンフィルタ"""
    # パディング
    pad = kernel_size // 2
    padded = np.pad(image, pad, mode='edge')
    
    # メディアンフィルタ適用
    result = np.zeros_like(image)
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            neighborhood = padded[i:i+kernel_size, j:j+kernel_size]
            result[i, j] = np.median(neighborhood)
    
    return result

# ノイズ画像を作成
noisy_img = add_noise(test_image, 30)

# メディアンフィルタでノイズ除去
denoised_img = median_filter(noisy_img)

# PSNRを計算
psnr_before = calculate_psnr(test_image, noisy_img)
psnr_after = calculate_psnr(test_image, denoised_img)

print(f"ノイズ追加前のPSNR: {psnr_before:.2f} dB")
print(f"ノイズ除去後のPSNR: {psnr_after:.2f} dB")

# 結果を可視化
plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.imshow(test_image, cmap='gray', vmin=0, vmax=255)
plt.title("オリジナル")
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(noisy_img, cmap='gray', vmin=0, vmax=255)
plt.title("ノイズ付き")
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(denoised_img, cmap='gray', vmin=0, vmax=255)
plt.title("ノイズ除去後")
plt.axis('off')

plt.tight_layout()
plt.show()

## まとめ

- 画像の統計量（平均、分散、ヒストグラムなど）を計算した
- 画像品質評価指標（PSNR, SSIM）を理解した
- ヒストグラム平坦化、フィルタリングなどの統計的処理を実践した

**次回は:** 画像セグメンテーション（閾値処理）について学びます